# 2_KNN_Diabetes_Classification

**Proyecto:** MLY0100 – Predicción de riesgo de Diabetes  
**Modelo:** K-Nearest Neighbors (KNN)  
**Autor:** Antonio Sepúlveda  
**Fecha:** 2025


## 1. Conexión con Kedro y carga de datos

En este notebook utilizamos el **pipeline Kedro** ya configurado.

- Cargamos el catalogo de datos con `%load_ext kedro.ipython` y `%reload_kedro`.  
- Leemos el dataset limpio `diabetes_cleaned` desde el `DataCatalog`.  
- La variable objetivo es **Outcome** (0 = No Diabetes, 1 = Diabetes).


In [ ]:
%load_ext kedro.ipython
%reload_kedro

# "context" y "catalog" quedan disponibles tras %reload_kedro
catalog.list()  # Para ver todos los datasets disponibles en el DataCatalog


In [ ]:
import pandas as pd

# Cargar el dataset limpio desde el catálogo de Kedro
df_diabetes = catalog.load("diabetes_cleaned")

print(df_diabetes.shape)
df_diabetes.head()

## 2. Importaciones

Importamos las librerías necesarias para:

- Manipulación y análisis de datos (`pandas`, `numpy`)  
- Visualización (`matplotlib`, `seaborn`)  
- Modelado y escalamiento (`KNeighborsClassifier`, `train_test_split`, `MinMaxScaler`)  
- Métricas de clasificación (accuracy, precision, recall, F1, matriz de confusión)  
- Curvas ROC y Precision-Recall  
- Búsqueda de hiperparámetros con `GridSearchCV`.


In [ ]:
# -- Tratamiento de datos --
import numpy as np

# -- Gráficos -- 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

# -- Procesado y modelado --
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# -- Métricas para modelos de clasificación --
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    auc,
    precision_recall_curve,
    PrecisionRecallDisplay,
    average_precision_score,
)

# -- GridSearchCV --
from sklearn.model_selection import GridSearchCV

sns.set(style="whitegrid")

## 3. Correlación inicial (opcional)

Revisamos la **matriz de correlación** para entender qué variables tienen mayor relación con el objetivo `Outcome`.

> Nota: este paso es exploratorio y no forma parte del pipeline de Kedro, pero ayuda a justificar las variables elegidas para el modelo KNN.


In [ ]:
numeric_df = df_diabetes.select_dtypes(include=np.number)
corr_matrix = numeric_df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriz de correlación – variables numéricas (diabetes)")
plt.tight_layout()
plt.show()


## 4. Desarrollo del modelo KNN

En esta sección entrenamos un modelo **K-Nearest Neighbors** para clasificar a los pacientes según la presencia de diabetes.

### 4.1 Selección de características

Para poder **visualizar la frontera de decisión**, utilizaremos solo **2 variables** como ejemplo:

- `Glucose`  
- `BMI`

La variable objetivo será `Outcome` (0 = No Diabetes, 1 = Diabetes).


In [ ]:
# Selección de características (solo 2 para poder graficar la frontera de decisión)
feature_cols = ["Glucose", "BMI"]
target_col = "Outcome"

X = df_diabetes[feature_cols].copy()
y = df_diabetes[target_col].copy()

print("Dimensiones de X:", X.shape)
print("Distribución de la variable objetivo:")
print(y.value_counts(normalize=True))


### 4.2 División de datos en entrenamiento y prueba

Dividimos el dataset en:

- **80% entrenamiento**  
- **20% prueba**  

Luego aplicamos **escalamiento Min-Max** para que KNN trabaje correctamente (es sensible a la escala de las variables).


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled[:5]

### 4.3 Creación y entrenamiento del modelo KNN

Comenzamos con un valor inicial de `k = 15` vecinos. Luego mejoraremos este valor con **GridSearchCV**.


In [ ]:
n_neighbors = 15

modelo_KNN = KNeighborsClassifier(n_neighbors=n_neighbors)
modelo_KNN.fit(X_train_scaled, y_train)

y_pred = modelo_KNN.predict(X_test_scaled)

print("Predicciones de ejemplo:", y_pred[:10])

### 4.4 Métricas de evaluación

Evaluamos el rendimiento del modelo con:

- **Accuracy**  
- **Precision**  
- **Recall (Sensibilidad)**  
- **F1-Score**  
- **Matriz de confusión**  
- **Especificidad** (Specificity) para el caso binario.


In [ ]:
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Matriz de confusión:\n", cm)
print("\nAccuracy:", acc)
print("Precision:", prec)
print("Recall (Sensibilidad):", rec)
print("F1-Score:", f1)

print("\nReporte de clasificación completo:\n")
print(classification_report(y_test, y_pred))

# Grafo de matriz de confusión
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Etiqueta predicha")
plt.ylabel("Etiqueta real")
plt.title("Matriz de confusión – KNN Diabetes")
plt.tight_layout()
plt.show()

# Sensibilidad y especificidad (solo binario)
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Sensibilidad (Recall): {sensitivity:.2f}")
print(f"Especificidad: {specificity:.2f}")

### 4.5 Frontera de decisión en 2D

Utilizando solo `Glucose` y `BMI` como características, graficamos la **frontera de decisión** del modelo KNN para entender cómo separa las clases.


In [ ]:
# Conversión a NumPy si todavía fuesen DataFrames
X_test_np = X_test_scaled
y_test_np = y_test.to_numpy().flatten()

# Crear malla
h = 0.02
x_min, x_max = X_test_np[:, 0].min() - 0.1, X_test_np[:, 0].max() + 0.1
y_min, y_max = X_test_np[:, 1].min() - 0.1, X_test_np[:, 1].max() + 0.1
xx, yy = np.meshgrid(
    np.arange(x_min, x_max, h),
    np.arange(y_min, y_max, h)
)

# Construir matriz de inputs para predecir
n_samples = xx.ravel().shape[0]
X_grid = np.c_[xx.ravel(), yy.ravel()]

# Predecir clases en la malla
Z = modelo_KNN.predict(X_grid)
Z = Z.reshape(xx.shape)

cmap_light = ListedColormap(["#FFAAAA", "#AAFFAA"])  # fondo
cmap_bold = ListedColormap(["#FF0000", "#00FF00"])   # puntos

plt.figure(figsize=(8, 6))
plt.pcolormesh(xx, yy, Z, cmap=cmap_light, shading="auto")
plt.scatter(
    X_test_np[:, 0],
    X_test_np[:, 1],
    c=y_test_np,
    cmap=cmap_bold,
    edgecolor="k",
    s=30,
)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xlabel("Glucose (escalado)")
plt.ylabel("BMI (escalado)")
plt.title(f"Frontera de decisión – KNN (k = {modelo_KNN.n_neighbors})")

patch_0 = mpatches.Patch(color="#FF0000", label="Clase 0 – No Diabetes")
patch_1 = mpatches.Patch(color="#00FF00", label="Clase 1 – Diabetes")
plt.legend(handles=[patch_0, patch_1])

plt.tight_layout()
plt.show()

### 4.6 Curva ROC

Calculamos la **curva ROC** y el **AUC** para evaluar el desempeño global del modelo en términos de tasa de falsos positivos y verdaderos positivos.


In [ ]:
y_pred_proba = modelo_KNN.predict_proba(X_test_scaled)[:, 1]

fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(7, 5))
plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("Tasa de Falsos Positivos")
plt.ylabel("Tasa de Verdaderos Positivos")
plt.title("Curva ROC – KNN Diabetes")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()


### 4.7 Curva Precision-Recall (PR)

La curva **Precision-Recall** es especialmente útil cuando la clase positiva (diabetes) está desbalanceada.


In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
ap_score = average_precision_score(y_test, y_pred_proba)

fig, ax = plt.subplots(figsize=(7, 5))
pr_display = PrecisionRecallDisplay(precision=precision, recall=recall)
pr_display.plot(ax=ax)
ax.set_title(f"Curva Precision-Recall – KNN (AP = {ap_score:.2f})")
plt.grid(True)
plt.tight_layout()
plt.show()


### 4.8 Búsqueda de hiperparámetros con GridSearchCV

Finalmente, usamos **GridSearchCV** para encontrar el mejor número de vecinos `k` en el rango de 1 a 30.

Luego evaluamos nuevamente el modelo óptimo en el conjunto de prueba.


In [ ]:
param_grid = {"n_neighbors": np.arange(1, 31)}

knn = KNeighborsClassifier()
grid_search = GridSearchCV(
    knn,
    param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
)

grid_search.fit(X_train_scaled, y_train)

print("Mejores parámetros:", grid_search.best_params_)
print("Mejor accuracy (cross-validation):", grid_search.best_score_)

best_knn = grid_search.best_estimator_
test_accuracy = best_knn.score(X_test_scaled, y_test)
print("Accuracy en test con mejores parámetros:", test_accuracy)

y_pred_gs = best_knn.predict(X_test_scaled)

print("\nReporte de clasificación (mejor modelo):\n")
print(classification_report(y_test, y_pred_gs))

cm_best = confusion_matrix(y_test, y_pred_gs)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_best, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de confusión – KNN (GridSearchCV)")
plt.tight_layout()
plt.show()

print("Accuracy Score (GridSearchCV):", accuracy_score(y_test, y_pred_gs))

# Sensibilidad y especificidad del mejor modelo
tn_b, fp_b, fn_b, tp_b = cm_best.ravel()
sensitivity_best = tp_b / (tp_b + fn_b)
specificity_best = tn_b / (tn_b + fp_b)

print(f"\nSensibilidad (Best Model): {sensitivity_best:.4f}")
print(f"Especificidad (Best Model): {specificity_best:.4f}")


### 4.9 Curvas ROC y PR para el mejor modelo

Repetimos el análisis de curvas ROC y Precision-Recall usando el **mejor modelo** encontrado por GridSearchCV.


In [ ]:
# Probabilidades para la clase positiva
y_pred_proba_best = best_knn.predict_proba(X_test_scaled)[:, 1]

# === Curva ROC ===
fpr_b, tpr_b, _ = roc_curve(y_test, y_pred_proba_best)
roc_auc_b = auc(fpr_b, tpr_b)

plt.figure(figsize=(7, 5))
plt.plot(fpr_b, tpr_b, color="darkorange", lw=2, label=f"ROC (AUC = {roc_auc_b:.2f})")
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("Falsos Positivos")
plt.ylabel("Verdaderos Positivos")
plt.title("Curva ROC – Mejor KNN (GridSearchCV)")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()

# === Curva Precision-Recall ===
precision_b, recall_b, _ = precision_recall_curve(y_test, y_pred_proba_best)
ap_b = average_precision_score(y_test, y_pred_proba_best)

plt.figure(figsize=(7, 5))
plt.plot(recall_b, precision_b, color="blue", lw=2, label=f"PR (AP = {ap_b:.2f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Curva Precision-Recall – Mejor KNN (GridSearchCV)")
plt.legend(loc="lower left")
plt.grid(True)
plt.tight_layout()
plt.show()
